In [ ]:
from collections.abc import Callable
import lmfit
import numpy as np
from qibo import gates
from qibo.models import Circuit
from qililab.automatic_calibration.utils.plotting import plot_iq
import qililab as ql
from qililab.platform import Platform
from qililab.utils import Wait
from tqdm import tqdm

In [ ]:
def plot_fit(func: Callable, xdata: np.ndarray, popt: list, ax, fitted_pi_pulse_amplitude):
    label_fit = f"FIT $A_\pi$ = {fitted_pi_pulse_amplitude:.3f}"
    ax.plot(xdata, func(xdata, *popt), "--", label=label_fit, color="red")
    ax.legend()

In [ ]:
def rabi(
    amp_values: np.ndarray,
    qubit: int,
    platform: Platform,
    hw_avg: int = 2_000,
    repetition_duration: int = 200_000,
    m_buffer_time: int = 0,
):
    # Define circuit
    circuit = Circuit(qubit + 1)
    circuit.add(ql.Drag(qubit, theta=np.pi, phase=0))
    circuit.add(Wait(qubit, m_buffer_time))
    circuit.add(gates.M(qubit))

    print(f"Running rabi on qubit {qubit}.")

    # Initial setup is needed to unsync all sequencers
    platform.initial_setup()

    # Run experiment
    results_list = []
    for amp in tqdm(iterable=amp_values, total=len(amp_values), desc="Amplitude"):
        platform.set_parameter(alias=f"Drag({qubit})", parameter=ql.Parameter.AMPLITUDE, value=float(amp))
        result = platform.execute(program=circuit, num_avg=hw_avg, repetition_duration=repetition_duration)
        results_list.append(result.array)

    return np.hstack(results_list)

In [ ]:
def fit_rabi(xdata: np.ndarray, results: np.ndarray, fit_quadrature: str = "i"):
    # Fitting function
    def sinus(x, a, b, c, d):
        return a * np.sin(2 * np.pi * b * np.array(x) - c) + d

    # Fit signal
    fit_signal = results[0] if fit_quadrature == "i" else results[1]

    # Sinus fit
    mod = lmfit.Model(sinus)
    mod.set_param_hint("a", value=0.3, max=3, vary=True)
    mod.set_param_hint("b", value=0.15, max=0.3, vary=True)
    mod.set_param_hint("c", value=0, max=np.pi, vary=True)
    mod.set_param_hint("d", value=0, max=10, vary=True)

    params = mod.make_params()
    fit = mod.fit(data=fit_signal, params=params, x=xdata)

    a_value = fit.params["a"].value
    b_value = fit.params["b"].value
    c_value = fit.params["c"].value
    d_value = fit.params["d"].value

    popt = [a_value, b_value, c_value, d_value]

    fitted_pi_pulse_amplitude = np.abs(1 / (2 * popt[1]))

    print(f"The fitted pi pulse amplitude is: {fitted_pi_pulse_amplitude}")
    print(popt)

    # plot
    fig, axes = plot_iq(xdata=xdata, results=results, title_label="Rabi", xlabel="Amplitude")
    plot_fit(
        func=sinus,
        xdata=np.linspace(xdata[0], xdata[-1], 1000),
        popt=popt,
        ax=axes[0 if fit_quadrature == "i" else 1],
        fitted_pi_pulse_amplitude=fitted_pi_pulse_amplitude,
    )
    return fitted_pi_pulse_amplitude, fig

In [ ]:
# Rabi experiment
rabi_values = {
    "start": 0,
    "stop": 0.25,
    "step": (0.25) / 50,  # It's written like this because it's derived from a np.linspace definition
}

qubit = 0
amp_values = np.arange(rabi_values["start"], rabi_values["stop"], rabi_values["step"])

In [ ]:
""" Define the platform and connect to the instruments """

os.environ["RUNCARDS"] = "./tests/automatic_calibration/runcards"
os.environ["DATA"] = "./tests/automatic_calibration/data"
platform_name = "galadriel"
platform_path = os.path.join(os.environ["RUNCARDS"], f"{platform_name}.yml")
platform = ql.build_platform(path=platform_path)

platform.connect()
platform.turn_on_instruments()
platform.initial_setup()

In [ ]:
results = rabi(amp_values=amp_values, qubit=qubit, platform=platform)

fitted_pi_pulse_amplitude, fig = fit_rabi(results=results)